In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders, text2tokens
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, EncoderDecoderAttn, DecoderAttnRNN, \
    DecoderDotAttnRNN
from train_eval import count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
from train_eval import train_and_eval
import importlib
import pickle as pkl 
import torch

In [2]:
# model identification
MODEL_NAME = 'zh-seq2seq-rnn-vanilla'
SRC_LANG = 'zh'
TARG_LANG = 'en'

# data processing params  
SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 #30000
TARG_VOCAB_SIZE = 30000 #30000

# model architecture params 
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 #2 
ENC_HIDDEN_DIM = 256 #512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM #2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 # to actually implement
DEC_DROPOUT = 0.2 # to actually implement
USE_ATTN = True

# training params  
BATCH_SIZE = 32 #32
NUM_EPOCHS = 200
LR = 0.0003 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = True

In [3]:
# store as dict to save to results later 
params = {'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 'rnn_cell_type': RNN_CELL_TYPE, 
          'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 
          'src_vocab_size': SRC_VOCAB_SIZE, 'targ_vocab_size': TARG_VOCAB_SIZE, 
          'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 'dec_hidden_dim': DEC_HIDDEN_DIM,
          'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 'clip_grad_max_norm': CLIP_GRAD_MAX_NORM,
          'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 'use_attn': USE_ATTN, 
          'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 'learning_rate': LR, 'optimizer': OPTIMIZER, 
          'lazy_train': LAZY_TRAIN} 

In [4]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [5]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, filter_long=False)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE, filter_long=False) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000, filter_long=False)

In [6]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [10]:
# define model 

encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=ENC_DROPOUT, 
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

# without attention 
decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
                     targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                     pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id'])

# with attention 
# decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
#                          num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
#                          targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
#                          pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']) 

# with dot attention 
decoder = DecoderDotAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                         num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
                         targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [11]:
model, results = train_and_eval(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=10000, save_checkpoint=True, save_to_log=True, 
    lazy_eval=False, print_attn=True, inspect_samples=1)

Epoch: 0.00, Train Loss: 10.16, Val Loss: 10.26, Train BLEU: 0.30, Val BLEU: 0.20, Minutes Elapsed: 0.05
Sampling from training predictions...
Source: 其实 它们 都 是 由 单独 的 动物 结合 合在
Reference: these are all individual animals banding together to make
Model: <SOS> it the the the the the the the the
Attention Weights: tensor([[3.9910e-05, 2.9069e-03, 1.0208e-01, 3.6750e-01, 4.0648e-01, 8.8430e-02,
         3.1263e-02, 1.2789e-03, 2.3951e-05, 9.3850e-08],
        [3.3833e-03, 2.6489e-02, 1.5200e-01, 2.8847e-01, 2.9621e-01, 1.3614e-01,
         7.9065e-02, 1.5985e-02, 2.1182e-03, 1.4055e-04],
        [2.2636e-02, 6.5902e-02, 1.6035e-01, 2.2654e-01, 2.2016e-01, 1.4209e-01,
         1.0183e-01, 4.2611e-02, 1.4208e-02, 3.6687e-03],
        [3.5100e-02, 7.6346e-02, 1.4396e-01, 1.8920e-01, 1.8635e-01, 1.4278e-01,
         1.1560e-01, 6.4330e-02, 3.2909e-02, 1.3419e-02],
        [3.6192e-02, 7.1498e-02, 1.2580e-01, 1.6447e-01, 1.6782e-01, 1.4454e-01,
         1.2899e-01, 8.2730e-02, 5.2829e-02, 2.513

Epoch: 3.00, Train Loss: 9.49, Val Loss: 9.98, Train BLEU: 0.28, Val BLEU: 0.19, Minutes Elapsed: 0.18
Sampling from training predictions...
Source: 大部 大部分 部分 的 动物 也 都 生活 在 海洋
Reference: most of the animals are in the oceans .
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[4.4584e-16, 2.4386e-11, 8.6877e-07, 2.9402e-03, 3.0255e-02, 7.6506e-01,
         2.0142e-01, 3.2837e-04, 5.6051e-07, 2.9604e-13],
        [1.9850e-10, 1.8560e-07, 1.4400e-04, 2.0886e-02, 8.5475e-02, 6.5590e-01,
         2.3399e-01, 3.5379e-03, 6.7646e-05, 7.9794e-09],
        [3.8896e-08, 7.0878e-06, 1.1895e-03, 4.6502e-02, 1.2730e-01, 5.9404e-01,
         2.2287e-01, 7.6905e-03, 4.0198e-04, 4.3180e-07],
        [4.1802e-07, 3.6872e-05, 3.0814e-03, 6.5660e-02, 1.4562e-01, 5.5369e-01,
         2.1953e-01, 1.1346e-02, 1.0307e-03, 3.5780e-06],
        [1.4691e-06, 8.8874e-05, 5.0601e-03, 7.8309e-02, 1.5344e-01, 5.2691e-01,
         2.2002e-01, 1.4316e-02, 1.8415e-03, 1.2878e-05],
        [3

Epoch: 6.00, Train Loss: 8.42, Val Loss: 9.54, Train BLEU: 0.28, Val BLEU: 0.19, Minutes Elapsed: 0.32
Sampling from training predictions...
Source: 泰坦 泰坦尼克 泰坦尼克号 坦尼 尼克 号 是 拿 了 不少
Reference: the truth of the matter is that the titanic
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[5.6471e-34, 3.1455e-34, 1.3270e-29, 2.5074e-25, 6.7483e-13, 1.3452e-04,
         4.0460e-01, 5.9519e-01, 7.3319e-05, 2.3104e-14],
        [5.2461e-23, 6.0788e-23, 2.7972e-20, 8.1447e-17, 2.6119e-08, 5.6843e-03,
         6.3813e-01, 3.5587e-01, 3.1553e-04, 7.4121e-11],
        [1.8207e-19, 2.1727e-19, 1.4556e-17, 3.5304e-14, 8.4146e-07, 2.0137e-02,
         7.2293e-01, 2.5660e-01, 3.3014e-04, 5.0103e-10],
        [6.2726e-18, 8.3116e-18, 1.9769e-16, 3.5868e-13, 2.8573e-06, 3.0305e-02,
         7.3599e-01, 2.3334e-01, 3.6601e-04, 1.3038e-09],
        [4.4454e-17, 6.3081e-17, 8.4591e-16, 1.2665e-12, 5.2419e-06, 3.6274e-02,
         7.3322e-01, 2.3007e-01, 4.3088e-04, 2.6254e-09],
  

Epoch: 10.00, Train Loss: 7.07, Val Loss: 9.02, Train BLEU: 0.28, Val BLEU: 0.19, Minutes Elapsed: 0.51
Sampling from training predictions...
Source: 我们 得用 非常 特殊 的 仪器 才能 能到 到达 那个
Reference: we have to have a very special technology to
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[8.3223e-38, 4.6517e-29, 4.5040e-12, 1.5797e-04, 9.6930e-01, 3.0542e-02,
         9.7546e-07, 5.6022e-19, 3.6370e-21, 6.4480e-38],
        [6.7508e-23, 4.6718e-17, 3.5667e-07, 7.3121e-03, 8.8302e-01, 1.0939e-01,
         2.7492e-04, 3.2939e-11, 3.1259e-13, 2.6438e-23],
        [5.6450e-19, 5.5130e-14, 7.0303e-06, 1.9870e-02, 8.4487e-01, 1.3423e-01,
         1.0313e-03, 2.0634e-09, 3.0165e-11, 2.1723e-19],
        [1.2495e-17, 6.3469e-13, 1.9726e-05, 2.8042e-02, 8.2681e-01, 1.4345e-01,
         1.6826e-03, 8.4942e-09, 1.5670e-10, 6.7912e-18],
        [4.2198e-17, 1.6827e-12, 2.9384e-05, 3.1978e-02, 8.1620e-01, 1.4967e-01,
         2.1246e-03, 1.6280e-08, 3.5117e-10, 3.6924e-17],
  

Epoch: 13.00, Train Loss: 6.24, Val Loss: 8.76, Train BLEU: 0.28, Val BLEU: 0.19, Minutes Elapsed: 0.67
Sampling from training predictions...
Source: 深海 海中 的 生命 大卫 <UNK> <EOS> <PAD> <PAD> <PAD>
Reference: life in the deep oceans <EOS> <PAD> <PAD> <PAD>
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0000, 0.0000, 0.9935, 0.0065, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.9124, 0.0876, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0002, 0.8733, 0.1265, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0004, 0.8516, 0.1480, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0004, 0.8336, 0.1660, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0004, 0.8214, 0.1782, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0004, 0.8138, 0.1858, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
    

...

Epoch: 183.00, Train Loss: 1.86, Val Loss: 11.82, Train BLEU: 33.55, Val BLEU: 0.29, Minutes Elapsed: 7.81
Sampling from training predictions...
Source: 当 你 站 在 海滩 上 或是 当 你 看到
Reference: part of the problem , i think , is
Model: <SOS> and of the the , the the the the
Attention Weights: tensor([[9.7466e-38, 2.0436e-14, 3.5897e-06, 5.9468e-03, 4.2955e-01, 3.5022e-01,
         2.1424e-01, 4.0727e-05, 1.7723e-10, 8.9372e-29],
        [2.8131e-29, 3.6409e-15, 3.7872e-08, 9.1433e-05, 6.3061e-01, 8.9605e-03,
         3.6032e-01, 1.9842e-05, 4.9465e-09, 4.9278e-18],
        [8.4919e-21, 5.0245e-10, 7.4254e-05, 4.1134e-03, 1.5336e-01, 3.8488e-01,
         4.5169e-01, 5.8685e-03, 4.0081e-06, 6.0032e-16],
        [2.0455e-19, 2.2030e-10, 8.7713e-05, 1.5136e-02, 1.5288e-01, 6.1650e-01,
         2.0715e-01, 8.2271e-03, 1.9209e-05, 2.7092e-13],
        [7.7795e-19, 4.4659e-11, 1.4187e-05, 1.4250e-02, 2.1696e-01, 5.9447e-01,
         1.7333e-01, 9.7642e-04, 3.9815e-07, 4.9449e-12],
        [2.0169e-1

Epoch: 186.00, Train Loss: 1.80, Val Loss: 11.67, Train BLEU: 33.95, Val BLEU: 0.28, Minutes Elapsed: 7.98
Sampling from training predictions...
Source: <UNK> 塞尔 <UNK> <UNK> 斯特 说 过 真正 的 探索
Reference: marcel proust said , &quot; the true voyage of
Model: <SOS> marcel proust said , &quot; the true voyage voyage
Attention Weights: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0001,
         0.9999],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

Epoch: 190.00, Train Loss: 1.71, Val Loss: 11.78, Train BLEU: 38.49, Val BLEU: 0.28, Minutes Elapsed: 8.19
Sampling from training predictions...
Source: 我 真 喜欢 这些 东西 <EOS> <PAD> <PAD> <PAD> <PAD>
Reference: i love that kind of stuff . <EOS> <PAD>
Model: <SOS> here &apos;s a kind about . . <EOS> <EOS>
Attention Weights: tensor([[0.0000, 0.0000, 0.9513, 0.0486, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1160, 0.5387, 0.0131, 0.3322, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0017, 0.6693, 0.3264, 0.0025, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0017, 0.0007, 0.9966, 0.0010, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
    

Epoch: 194.00, Train Loss: 1.63, Val Loss: 11.76, Train BLEU: 43.37, Val BLEU: 0.29, Minutes Elapsed: 8.40
Sampling from training predictions...
Source: 这 是 一种 种群 栖 动物 <EOS> <PAD> <PAD> <PAD>
Reference: it &apos;s a colonial animal . <EOS> <PAD> <PAD>
Model: <SOS> here &apos;s a colonial animal . <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.0000, 0.0000, 0.2318, 0.7682, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1956, 0.8039, 0.0000, 0.0000, 0.0004, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0005, 0.6495, 0.3500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0037, 0.9739, 0.0200, 0.0000, 0.0024, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000

Epoch: 198.00, Train Loss: 1.56, Val Loss: 11.79, Train BLEU: 46.07, Val BLEU: 0.28, Minutes Elapsed: 8.58
Sampling from training predictions...
Source: 其实 它们 都 是 由 单独 的 动物 结合 合在
Reference: these are all individual animals banding together to make
Model: <SOS> these are all individual animals banding together to make
Attention Weights: tensor([[0.0000, 0.0002, 0.0002, 0.0034, 0.0031, 0.9931, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.0000, 0.0001, 0.0002, 0.0068, 0.0121, 0.9807, 0.0001, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0091, 0.0015, 0.0168, 0.0222, 0.9504, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0395, 0.0021, 0.0530, 0.0863, 0.8190, 0.0001, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0001, 0.0000, 0.0002, 0.0003, 0.0018, 0.0000, 0.0000, 0.0000,
         0.9976],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

In [ ]:
summarize_results(load_experiment_log())[['dt_created', 'num_epochs', 'learning_rate', 'clip_grad_max_norm', 'val_loss']].head()

In [ ]:
plot_single_learning_curve(results)

In [ ]:
# Epoch: 199.00, Train Loss: 0.32, Val Loss: 13.19, Train BLEU: 98.94, Val BLEU: 0.27
plot_single_learning_curve(results)

In [ ]:
# with attention energies = v_broadcast.bmm(torch.tanh(self.attn(concat)).transpose(1, 2)) # switched order  
# Epoch: 199.00, Train Loss: 0.63, Val Loss: 12.82, Train BLEU: 92.05, Val BLEU: 0.38
plot_single_learning_curve(results)

In [ ]:
for i, token in enumerate(vocab[SRC_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
for i, token in enumerate(vocab[TARG_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
import torch
x = torch.arange(0, 3*5*10).view(3, 5, 10)
print(x)
y = x[1:, :, :]
print(y)
z = y.view(-1, 10)
print(z)

In [ ]:
t = torch.arange(0, 2*5).view(5, 2)
print(t)
u = t.contiguous().view(-1)
print(u)
v = t.permute(1, 0)
print(v)
w = v.contiguous().view(-1)
print(w)

In [ ]:
a = torch.arange(0, 2*1*300)
print(a)
b = a.view(-1, 1, 300)
print(b.size())

In [ ]:
for i, (src_idxs, targ_idxs, src_lens, targ_lens) in enumerate(loaders_full['train']):
#     print(i)
#     print(src_idxs.size())
#     print(src_idxs)
#     print(src_lens)
#     print(targ_idxs.size())
#     print(targ_idxs)
#     print(targ_lens)
    id2token = vocab[SRC_LANG]['id2token']
    test_tensor = src_idxs
    list_of_lists = test_tensor.numpy().astype(int).tolist()
    to_token = lambda l: ' '.join([id2token[idx] for idx in l])
    list_of_lists_tokens = [to_token(l) for l in list_of_lists] 
    break 

In [ ]:
#attn.data.masked_fill_(self.mask, -float('inf'))
test_tensor.

test_tensor.data.masked_fill_(test_tensor == 2, float('inf'))